In [1]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel

import time
import pickle
from tqdm import tqdm

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\Chankoo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
# with open('artNouns.dat','rb') as file: # load
#     nouns = pickle.load(file)

In [2]:
with open('artAd_nouns.dat','rb') as file: # load
    ad_nouns = pickle.load(file)

전처리해주자 

TFIDF 기준으로 0.4 미만 단어는 불용어처리

In [3]:
with open('TFIDF.dic','rb') as file:
    TFIDF = pickle.load(file)

In [4]:
ad_nouns_refined = []
for i, doc in enumerate(ad_nouns):
    tmp = []
    for token in doc:
        if (TFIDF[i][token] >0.4):
            tmp.append(token)
    ad_nouns_refined.append(tmp)

In [5]:
dic = corpora.Dictionary(ad_nouns_refined)

In [27]:
type(dic)

gensim.corpora.dictionary.Dictionary

In [11]:
len(dic) #출현 단어 개수

82299

In [12]:
corpus = [dic.doc2bow(doc) for doc in ad_nouns_refined] #단어별 출현 횟수 vector 구성

In [20]:
s = time.time()
ldamodel = gensim.models.ldamodel.LdaModel(
                        corpus, 
                        num_topics=15, 
                        id2word = dic, 
                        passes=10
                        ) #LDA model 생성
print(time.time()-s)

322.3346176147461


In [14]:
ldamodel.save('lda_15.model')

In [15]:
ldamodel = gensim.models.ldamodel.LdaModel.load('lda_15.model')

In [26]:
ldamodel.show_topics(num_words=30) #토픽을 구성하는 주요단어들

[(0,
  '0.021*"조각" + 0.013*"재료" + 0.008*"오브" + 0.007*"평면" + 0.006*"입체" + 0.006*"종이" + 0.005*"조형" + 0.005*"구조" + 0.004*"설치" + 0.004*"드로잉" + 0.004*"제작" + 0.004*"디자인" + 0.004*"기법" + 0.004*"흙" + 0.004*"표면" + 0.004*"공예" + 0.004*"나무" + 0.003*"만든" + 0.003*"유리" + 0.003*"가구" + 0.003*"물질" + 0.003*"물성" + 0.003*"차원" + 0.003*"제" + 0.003*"금속" + 0.003*"확장" + 0.003*"실험" + 0.003*"손" + 0.003*"캔버스" + 0.003*"건축"'),
 (1,
  '0.016*"빛" + 0.013*"색채" + 0.011*"색" + 0.008*"풍경" + 0.006*"산" + 0.005*"기법" + 0.005*"추상" + 0.005*"감성" + 0.005*"캔버스" + 0.004*"붓" + 0.004*"묘사" + 0.004*"조형" + 0.004*"산수" + 0.004*"물감" + 0.004*"구성" + 0.004*"화가" + 0.004*"감각" + 0.004*"그려" + 0.003*"효과" + 0.003*"재현" + 0.003*"그린" + 0.003*"조화" + 0.003*"내면" + 0.003*"그림자" + 0.003*"배경" + 0.003*"여백" + 0.003*"산수화" + 0.003*"풍경화" + 0.003*"정서" + 0.003*"평면"'),
 (3,
  '0.026*"기억" + 0.023*"집" + 0.022*"도시" + 0.013*"장소" + 0.010*"과거" + 0.010*"건물" + 0.008*"흔적" + 0.007*"기록" + 0.006*"추억" + 0.006*"서울" + 0.005*"벽" + 0.005*"마을" + 0.005*"방" + 0.005*"시절" + 0.005*"오래된" + 0

#### 참고
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling

In [27]:
# Compute Perplexity k=15:-8.982 , k=10: -8.9, k=7: -8.989
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -8.982701484810478


In [28]:
# Compute Coherence Score k=15:0.48977129725648366 ,k=10:0.467156822233572, k=7:0.47333799087547396
coherence_model_lda = CoherenceModel(model=ldamodel, texts=ad_nouns_refined, dictionary=dic, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.48977129725648366


In [29]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dic)
vis

C:\Users\Chankoo\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.068946 -0.025325       1        1  13.615334
14    -0.035106 -0.093224       2        1  13.409978
7     -0.143014  0.160933       3        1   9.507203
0      0.024916 -0.202964       4        1   7.206053
13     0.267664 -0.017644       5        1   6.935299
1     -0.089911 -0.200023       6        1   6.617240
9      0.179661 -0.067352       7        1   6.545925
5     -0.116382  0.006423       8        1   6.335234
11    -0.020519  0.001047       9        1   6.154649
6     -0.166421 -0.017739      10        1   5.399288
2     -0.071076  0.175100      11        1   4.172282
10    -0.137407 -0.072777      12        1   3.997851
8      0.223193 -0.042726      13        1   3.674413
3     -0.077397  0.193892      14        1   3.427928
12     0.230744  0.202379      15        1   3.001324, topic_info=     Category          Freq  Term         Total  loglift  logprob
term                                                             
744   Default  11280.000000    사진  11280.000000  30.0000  30.0000
874   Default   8958.000000     꽃   8958.000000  29.0000  29.0000
132   Default  10154.000000    도시  10154.000000  28.0000  28.0000
951   Default   8412.000000    조각   8412.000000  27.0000  27.0000
2346  Default   4526.000000    기억   4526.000000  26.0000  26.0000
48    Default   6932.000000    역사   6932.000000  25.0000  25.0000
1790  Default   7326.000000     빛   7326.000000  24.0000  24.0000
544   Default   5211.000000     집   5211.000000  23.0000  23.0000
939   Default   6581.000000    욕망   6581.000000  22.0000  22.0000
30    Default   5811.000000   미술관   5811.000000  21.0000  21.0000
188   Default   5588.000000    생명   5588.000000  20.0000  20.0000
2133  Default   4473.000000    얼굴   4473.000000  19.0000  19.0000
349   Default   7346.000000    나무   7346.000000  18.0000  18.0000
52    Default   4285.000000     월   4285.000000  17.0000  17.0000
225   Default   4263.000000    영상   4263.000000  16.0000  16.0000
1058  Default   6277.000000    재료   6277.000000  15.0000  15.0000
1179  Default   5492.000000    장소   5492.000000  14.0000  14.0000
496   Default   3895.000000     색   3895.000000  13.0000  13.0000
3187  Default   5286.000000     꿈   5286.000000  12.0000  12.0000
1278  Default   5713.000000    색채   5713.000000  11.0000  11.0000
245   Default   6394.000000    인물   6394.000000  10.0000  10.0000
949   Default   5347.000000    전통   5347.000000   9.0000   9.0000
789   Default   4641.000000    입니   4641.000000   8.0000   8.0000
1746  Default   4681.000000    아트   4681.000000   7.0000   7.0000
335   Default   5506.000000    흔적   5506.000000   6.0000   6.0000
144   Default   4870.000000    매체   4870.000000   5.0000   5.0000
1541  Default   4281.000000   아름다   4281.000000   4.0000   4.0000
345   Default   4029.000000    과거   4029.000000   3.0000   3.0000
14    Default   4081.000000    기획   4081.000000   2.0000   2.0000
1657  Default   3926.000000  프로젝트   3926.000000   1.0000   1.0000
...       ...           ...   ...           ...      ...      ...
2226  Topic15    511.364532   아시아    728.502197   3.1522  -5.7094
4854  Topic15    311.383850    국민    401.271790   3.2525  -6.2054
5579  Topic15    178.027451    군인    202.578964   3.3769  -6.7645
3788  Topic15    197.956467   올림픽    234.500153   3.3367  -6.6584
2747  Topic15    302.633331    민중    426.008240   3.1642  -6.2339
5638  Topic15    528.152405    경제   1027.359863   2.8408  -5.6771
6223  Topic15    186.997101    정부    231.420334   3.2930  -6.7154
1824  Topic15    716.246155    중국   1779.583496   2.5960  -5.3724
2546  Topic15    530.270020    나라   1263.609619   2.6378  -5.6731
8391  Topic15    230.096054    혁명    326.447296   3.1563  -6.5080
8148  Topic15    379.818329    인도    785.554382   2.7794  -6.0068
4074  Topic15    483.894226    지구   1205.594971   2.5933  -5.7646
3078  Topic15    644.689636    미국   2117.263184   2.3170  -5.4777
919

![1](https://user-images.githubusercontent.com/38183218/44544400-0ef41e00-a74d-11e8-894d-0260f2672cc9.PNG)

#### LDA_artists

In [30]:
with open('artists_twice_tokens.lst','rb') as file:
    artists_twice_tokens = pickle.load(file)

In [32]:
with open('TFIDF_artists.dic','rb') as file:
    TFIDF = pickle.load(file)

In [31]:
len(artists_twice_tokens)

7565

불용어처리 느낌으로 TFIDF 0.2 미만 단어는 제외한다

In [33]:
artists_twice_tokens_refined = []
for tpl in artists_twice_tokens:
    artist_name = tpl[0]
    tokens = tpl[1]
    
    tmp = []
    for token in tokens:
        if (TFIDF[artist_name][token] >0.2):
            tmp.append(token)
    artists_twice_tokens_refined.append(tmp)

In [34]:
dic = corpora.Dictionary(artists_twice_tokens_refined)

In [35]:
len(dic) #출현 단어 개수

91901

In [36]:
corpus = [dic.doc2bow(doc) for doc in ad_nouns_refined] #단어별 출현 횟수 vector 구성

In [37]:
%%time
ldamodel = gensim.models.ldamodel.LdaModel(
                        corpus, 
                        num_topics=10, 
                        id2word = dic, 
                        passes=10
                        ) #LDA model 생성

Wall time: 4min 39s


In [38]:
ldamodel.show_topics(num_words=30) #토픽을 구성하는 주요단어들

[(0,
  '0.009*"꿈" + 0.006*"아이" + 0.005*"동물" + 0.005*"사랑" + 0.005*"현대인" + 0.005*"어린" + 0.004*"상상" + 0.004*"행복" + 0.004*"시절" + 0.004*"살아가는" + 0.004*"집" + 0.004*"희망" + 0.004*"추억" + 0.004*"소통" + 0.004*"가족" + 0.003*"여행" + 0.003*"누구" + 0.003*"치유" + 0.003*"본인" + 0.003*"동화" + 0.003*"느끼" + 0.003*"캐릭터" + 0.003*"작은" + 0.002*"인형" + 0.002*"표현하고" + 0.002*"인생" + 0.002*"나를" + 0.002*"노트" + 0.002*"상처" + 0.002*"감성"'),
 (1,
  '0.005*"그런" + 0.004*"그녀" + 0.004*"그래서" + 0.004*"이런" + 0.004*"였" + 0.003*"손" + 0.003*"좋" + 0.003*"않았" + 0.003*"그렇게" + 0.003*"않" + 0.003*"이제" + 0.003*"날" + 0.003*"같다" + 0.003*"일이" + 0.003*"그렇" + 0.003*"이었" + 0.002*"이유" + 0.002*"저" + 0.002*"정도" + 0.002*"화가" + 0.002*"이름" + 0.002*"얼굴" + 0.002*"잘" + 0.002*"없었" + 0.002*"처음" + 0.002*"않다" + 0.002*"그리" + 0.002*"누구" + 0.002*"몇" + 0.002*"수도"'),
 (2,
  '0.016*"꽃" + 0.008*"색채" + 0.007*"색" + 0.006*"생명" + 0.006*"빛" + 0.005*"나무" + 0.005*"화가" + 0.005*"아름다" + 0.004*"감성" + 0.004*"정신" + 0.003*"붓" + 0.003*"기법" + 0.003*"물감" + 0.003*"그려" + 0.003*"추상" + 0.00

In [39]:
# Compute Perplexity 
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -8.945683322131323


In [40]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=ad_nouns_refined, dictionary=dic, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.472963885369743


In [41]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dic)
vis

C:\Users\Chankoo\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.017398 -0.135715       1        1  21.339979
9      0.138615 -0.094559       2        1  15.379700
1      0.117196  0.054244       3        1  11.418814
6     -0.255688 -0.051830       4        1  11.412531
2      0.030045  0.094701       5        1  11.307054
0      0.082788  0.047667       6        1   9.219865
3      0.047830 -0.209287       7        1   6.577271
8     -0.009469 -0.088117       8        1   5.555391
4     -0.268651  0.143952       9        1   4.102097
5      0.134733  0.238945      10        1   3.687296, topic_info=      Category          Freq  Term         Total  loglift  logprob
term                                                              
53579  Default  11510.000000    사진  11510.000000  30.0000  30.0000
1596   Default   9987.000000    도시   9987.000000  29.0000  29.0000
1148   Default   9341.000000     꽃   9341.000000  28.0000  28.0000
4183   Default   4474.000000    입니   4474.000000  27.0000  27.0000
1196   Default   7391.000000    나무   7391.000000  26.0000  26.0000
67329  Default   6053.000000   미술관   6053.000000  25.0000  25.0000
49673  Default   4853.000000    전통   4853.000000  24.0000  24.0000
3722   Default   6541.000000    욕망   6541.000000  23.0000  23.0000
5339   Default   3761.000000    하늘   3761.000000  22.0000  22.0000
2714   Default   3973.000000     산   3973.000000  21.0000  21.0000
2812   Default   5641.000000    색채   5641.000000  20.0000  20.0000
36076  Default   4913.000000    아트   4913.000000  19.0000  19.0000
5401   Default   3023.000000     합   3023.000000  18.0000  18.0000
1161   Default   5304.000000     꿈   5304.000000  17.0000  17.0000
3515   Default   3806.000000    여성   3806.000000  16.0000  16.0000
4283   Default   5381.000000    장소   5381.000000  15.0000  15.0000
60456  Default   4538.000000    기억   4538.000000  14.0000  14.0000
68766  Default   2567.000000    풍경   2567.000000  13.0000  13.0000
2218   Default   2667.000000    바다   2667.000000  12.0000  12.0000
2804   Default   3788.000000     색   3788.000000  11.0000  11.0000
1030   Default   4938.000000    기법   4938.000000  10.0000  10.0000
78974  Default   6852.000000    역사   6852.000000   9.0000   9.0000
1020   Default   4478.000000    기록   4478.000000   8.0000   8.0000
9161   Default   3275.000000    바람   3275.000000   7.0000   7.0000
5301   Default   4088.000000  프로젝트   4088.000000   6.0000   6.0000
2860   Default   4114.000000    서울   4114.000000   5.0000   5.0000
3324   Default   3431.000000    아이   3431.000000   4.0000   4.0000
48211  Default   3559.000000    사물   3559.000000   3.0000   3.0000
3085   Default   3144.000000     숲   3144.000000   2.0000   2.0000
2640   Default   7207.000000     빛   7207.000000   1.0000   1.0000
...        ...           ...   ...           ...      ...      ...
6117   Topic10    217.164047    계곡    244.308609   3.1825  -6.7561
2235   Topic10    567.867004    바위    715.608521   3.0690  -5.7949
2714   Topic10   2536.843750     산   3973.322266   2.8516  -4.2981
1752   Topic10   1486.113525     땅   2348.454834   2.8427  -4.8329
1617   Topic10   1176.266724     돌   1822.255371   2.8625  -5.0667
9161   Topic10   1837.229492    바람   3275.508545   2.7221  -4.6208
4458   Topic10    327.739868    정선    418.851807   3.0550  -6.3446
21078  Topic10    270.249695    호수    334.385925   3.0873  -6.5374
2719   Topic10    826.052063    산수   1383.353394   2.7847  -5.4201
2220   Topic10    172.921722   바닷가    197.054565   3.1696  -6.9839
1877   Topic10    785.900879    마을   1437.594727   2.6964  -5.4699
82196  Topic10    759.331482     길   1380.149170   2.7028  -5.5043
3344   Topic10    378.168030    안개    564.858032   2.8990  -6.2014
1196   Topic10   2566.453125    나무   7391.153320   2.2425  -4.2865
3085   Topic10   1289.988403     숲   3144.609863   2.4092  -4.9744
768    Topic10    650.152466    고향   1244.778564   2.6508  -5.6596
4468   Topic10    601.3966